In [1]:
import torch
from models import ConvAttnNet
from models import resnet
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from Datasets.datasets import MyDataset

import ops.schedulers as schedulers
import ops.trains as trains
import ops.tests as tests
# import ops.datasets as datasets


import models
import datetime
import os
import yaml
import copy

In [2]:
dataset_train = MyDataset('E:/datasets/imagenet-mini/train/trainset.csv')
dataset_test = MyDataset('E:/datasets/imagenet-mini/val/valset.csv')

In [3]:
dataset_train = DataLoader(dataset_train, 
                           shuffle=True, 
                           num_workers=2, 
                           batch_size=128)
dataset_test = DataLoader(dataset_test, 
                          num_workers=2, 
                          batch_size=128)

print("Train: %s, Test: %s, Classes: %s" % (
    len(dataset_train.dataset), 
    len(dataset_test.dataset), 
    1000
))

Train: 34745, Test: 3923, Classes: 1000


In [2]:
model = ConvAttnNet.dnn_34(num_classes=2)

D:\anaconda\envs\pytorch19\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
dataset_name = "imagenet_mini"

In [5]:
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = os.path.join("runs", dataset_name, model.name, current_time)
writer = SummaryWriter(log_dir)

# with open("%s/config.yaml" % log_dir, "w") as f:
#     yaml.dump(args, f)
with open("%s/model.log" % log_dir, "w") as f:
    f.write(repr(model))

print("Create TensorBoard log dir: ", log_dir)

Create TensorBoard log dir:  runs\imagenet_mini\ConvAttnNet_34\20220728_091433


In [8]:
optim_args = {'name': 'SGD', 'lr': 1.25e-04, 'momentum': 0.9, 'nesterov': False, 'weight_decay': 0.0005, 
              'scheduler': {'name': 'MultiStepLR', 'milestones': [60, 120, 160], 'gamma': 0.2}}
train_args = {'warmup_epochs': 1, 'epochs': 200, 'batch_size': 128}
val_args = {'batch_size': 256, 'n_ff': 1}

In [9]:
torch.cuda.set_device(0)

In [10]:
gpu = torch.cuda.is_available()
optimizer, train_scheduler = trains.get_optimizer(model, **optim_args)
warmup_scheduler = schedulers.WarmupScheduler(optimizer, len(dataset_train) * train_args.get("warmup_epochs", 0))

In [ ]:
trains.train(model, optimizer,
             dataset_train, dataset_test,
             train_scheduler, warmup_scheduler,
             train_args, val_args, gpu,
             writer, 
             snapshot=2, dataset_name=dataset_name, uid=current_time)  # Set `snapshot=N` to save snapshots every N epochs.

(205.81 sec/epoch) Warmup epoch: 0, Loss: 6.9413, lr: 1.250e-04
NLL: 6.9451, Cutoffs: 0.0 %, 90.0 %, Accs: 0.178 %, 0.000 %, Uncs: 0.000 %, 100.000 %, IoUs: 0.018 %, 0.000 %, Freqs: 100.000 %, 0.000 %, Top-5: 0.561 %, Brier: 0.999, ECE: 0.052 %, ECE±: 0.052 %
The model is warmed up: 241.63 sec 

(98.05 sec/epoch) Epoch: 0, Loss: 6.9229, lr: 1.250e-04
NLL: 6.9276, Cutoffs: 0.0 %, 90.0 %, Accs: 0.102 %, 0.000 %, Uncs: 0.000 %, 100.000 %, IoUs: 0.001 %, 0.000 %, Freqs: 100.000 %, 0.000 %, Top-5: 0.433 %, Brier: 0.999, ECE: 0.126 %, ECE±: 0.126 %
(97.92 sec/epoch) Epoch: 1, Loss: 6.9020, lr: 1.250e-04
NLL: 6.9121, Cutoffs: 0.0 %, 90.0 %, Accs: 0.306 %, 0.000 %, Uncs: 0.000 %, 100.000 %, IoUs: 0.025 %, 0.000 %, Freqs: 100.000 %, 0.000 %, Top-5: 0.663 %, Brier: 0.999, ECE: 0.095 %, ECE±: 0.054 %
(97.99 sec/epoch) Epoch: 2, Loss: 6.8803, lr: 1.250e-04
NLL: 6.8942, Cutoffs: 0.0 %, 90.0 %, Accs: 0.357 %, 0.000 %, Uncs: 0.000 %, 100.000 %, IoUs: 0.053 %, 0.000 %, Freqs: 100.000 %, 0.000 %, Top-5